Классификация текста

In [0]:
import torch
import random
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
SEED = 0

In [0]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
!pip install transformers

     |████████████████████████████████| 645kB 8.4MB/s 
     |████████████████████████████████| 1.1MB 24.2MB/s 
     |████████████████████████████████| 890kB 54.3MB/s 
     |████████████████████████████████| 3.8MB 54.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1c8aeab4d91e5e08166fd2de892f6d618c6bc0edb041dbc7e0704dab4cf42207
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
len(tokenizer.vocab)

30522

In [0]:
print(tokenizer.tokenize("I create my BERT model"))

['i', 'create', 'my', 'bert', 'model']


In [0]:
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I create my BERT model")))

[1045, 3443, 2026, 14324, 2944]


In [0]:
CLS = tokenizer.cls_token_id
SEP = tokenizer.sep_token_id
PAD = tokenizer.pad_token_id
UNK = tokenizer.unk_token_id

In [0]:
print(tokenizer.max_model_input_sizes)

{'bert-base-uncased': 512, 'bert-large-uncased': 512, 'bert-base-cased': 512, 'bert-large-cased': 512, 'bert-base-multilingual-uncased': 512, 'bert-base-multilingual-cased': 512, 'bert-base-chinese': 512, 'bert-base-german-cased': 512, 'bert-large-uncased-whole-word-masking': 512, 'bert-large-cased-whole-word-masking': 512, 'bert-large-uncased-whole-word-masking-finetuned-squad': 512, 'bert-large-cased-whole-word-masking-finetuned-squad': 512, 'bert-base-cased-finetuned-mrpc': 512, 'bert-base-german-dbmdz-cased': 512, 'bert-base-german-dbmdz-uncased': 512, 'bert-base-finnish-cased-v1': 512, 'bert-base-finnish-uncased-v1': 512, 'bert-base-dutch-cased': 512}


In [0]:
def sent2toks(sent):
  toks = tokenizer.tokenize(sent)
  toks = toks[:int(tokenizer.max_model_input_sizes['bert-base-uncased'] / 2 - 2)]
  return toks

In [0]:
sent2toks('I create my BERT model')

['i', 'create', 'my', 'bert', 'model']

In [0]:
!pip install torchtext

In [0]:
from torchtext import data
from torchtext import datasets

In [0]:
TEXT = data.Field(batch_first=True, use_vocab=False, tokenize=sent2toks, preprocessing=tokenizer.convert_tokens_to_ids, init_token=CLS, eos_token=SEP, pad_token=PAD, unk_token=UNK)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
data_train, data_val, data_test = datasets.SNLI.splits(text_field=TEXT, label_field=LABEL)

downloading snli_1.0.zip


snli_1.0.zip: 100%|██████████| 94.6M/94.6M [01:10<00:00, 1.34MB/s]


extracting


In [0]:
print(len(data_train), len(data_val), len(data_test))
print(vars(data_train.examples[0]))
print('premise: ', tokenizer.convert_ids_to_tokens(vars(data_train.examples[0])['premise']))
print('hypothesis: ', tokenizer.convert_ids_to_tokens(vars(data_train.examples[0])['hypothesis']))
print('label: ', vars(data_train.examples[0])['label'])

549367 9842 9824
{'premise': [1037, 2711, 2006, 1037, 3586, 14523, 2058, 1037, 3714, 2091, 13297, 1012], 'hypothesis': [1037, 2711, 2003, 2731, 2010, 3586, 2005, 1037, 2971, 1012], 'label': 'neutral'}
premise:  ['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.']
hypothesis:  ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.']
label:  neutral


In [0]:
example = data_train.examples[0]
print(example)

In [0]:
LABEL.build_vocab(data_train)
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7ff0dfe77510>, {'entailment': 0, 'contradiction': 1, 'neutral': 2})


In [0]:
BATCH_SIZE = 256
train_it, val_it, test_it = data.BucketIterator.splits((data_train, data_val, data_test), batch_sizes=[BATCH_SIZE, BATCH_SIZE // 2, BATCH_SIZE // 2], device=device)

In [0]:
batch_0 = ''
for i, batch in enumerate(train_it):
  batch_0 = batch

In [0]:
batch_0.premise.shape

torch.Size([256, 51])

In [0]:
torch.cat((batch_0.premise, batch_0.hypothesis), dim=1).shape

torch.Size([256, 82])

In [0]:
lbl = torch.zeros((len(batch_0.label), len(LABEL.vocab)))
for i in range(batch_0.label.shape[0]):
  lbl[i][int(batch_0.label[i])] = 1
print(batch_0.label, lbl)

tensor([2., 0., 2., 0., 2., 0., 2., 1., 1., 2., 2., 0., 2., 2., 2., 2., 0., 0.,
        1., 1., 2., 1., 2., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 2., 0.,
        1., 1., 2., 0., 1., 2., 2., 0., 1., 1., 1., 0., 1., 1., 1., 2., 0., 0.,
        1., 1., 1., 2., 1., 0., 2., 2., 1., 2., 2., 1., 2., 0., 1., 0., 1., 2.,
        2., 2., 2., 2., 1., 0., 1., 2., 0., 2., 2., 1., 0., 0., 0., 1., 1., 2.,
        0., 1., 0., 0., 1., 0., 2., 2., 0., 1., 1., 2., 2., 0., 2., 2., 2., 0.,
        1., 1., 2., 2., 0., 0., 2., 1., 1., 2., 1., 2., 2., 2., 0., 1., 2., 1.,
        0., 0., 1., 2., 1., 2., 1., 1., 2., 1., 2., 2., 2., 1., 0., 2., 1., 2.,
        2., 1., 1., 2., 1., 0., 1., 1., 2., 2., 1., 2., 2., 0., 0., 0., 0., 0.,
        2., 1., 2., 2., 0., 0., 2., 2., 0., 1., 0., 0., 1., 2., 0., 1., 2., 0.,
        2., 1., 1., 0., 1., 2., 0., 1., 2., 0., 1., 1., 1., 2., 2., 1., 0., 0.,
        2., 0., 1., 0., 1., 0., 2., 1., 0., 2., 2., 2., 1., 1., 2., 1., 1., 1.,
        2., 1., 0., 2., 0., 1., 1., 1., 

In [0]:
from transformers import BertModel

In [0]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [0]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
import torch.nn as nn
import torch.nn.functional as func

In [0]:
class EClassifier(nn.Module):
  def __init__(self, bert, hid_dim, out_dim, num_lay, bi, drop):
    super().__init__()

    self.inp2emb = bert
    self.emb2hid = nn.GRU(bert.config.to_dict()['hidden_size'], hid_dim, num_layers=num_lay, bidirectional=bi, batch_first=True, dropout=drop)
    self.hid2out = nn.Linear(hid_dim * 2 if bi else hid_dim, out_dim)
    self.drop = nn.Dropout(drop)

  def forward(self, inp_pre, inp_hyp):
    with torch.no_grad():
      emb = self.inp2emb(torch.cat((inp_pre, inp_hyp), dim=1))[0]
    
    hid, h = self.emb2hid(emb)
    
    if self.emb2hid.bidirectional:
      h = self.drop(torch.cat((h[-2,:,:], h[-1,:,:]), dim=1))
    else:
      h = self.drop(h[-1,:,:])
    
    out = self.hid2out(h)
    out = func.softmax(out, dim = 1)
    return out

In [0]:
HID_DIM = 256
OUT_DIM = len(LABEL.vocab)
NUM_LAY = 3
BI = True
DROP = 0.25

eclassifier = EClassifier(bert, HID_DIM, OUT_DIM, NUM_LAY, BI, DROP)

In [0]:
sum(p.numel() for p in eclassifier.parameters() if p.requires_grad)

113425155

In [0]:
for name, param in eclassifier.named_parameters():
  if name.startswith('inp2emb'):
    param.requires_grad = False

In [0]:
sum(p.numel() for p in eclassifier.parameters() if p.requires_grad)

3942915

In [0]:
for name, param in eclassifier.named_parameters():
  if param.requires_grad:
    print(name)

emb2hid.weight_ih_l0
emb2hid.weight_hh_l0
emb2hid.bias_ih_l0
emb2hid.bias_hh_l0
emb2hid.weight_ih_l0_reverse
emb2hid.weight_hh_l0_reverse
emb2hid.bias_ih_l0_reverse
emb2hid.bias_hh_l0_reverse
emb2hid.weight_ih_l1
emb2hid.weight_hh_l1
emb2hid.bias_ih_l1
emb2hid.bias_hh_l1
emb2hid.weight_ih_l1_reverse
emb2hid.weight_hh_l1_reverse
emb2hid.bias_ih_l1_reverse
emb2hid.bias_hh_l1_reverse
emb2hid.weight_ih_l2
emb2hid.weight_hh_l2
emb2hid.bias_ih_l2
emb2hid.bias_hh_l2
emb2hid.weight_ih_l2_reverse
emb2hid.weight_hh_l2_reverse
emb2hid.bias_ih_l2_reverse
emb2hid.bias_hh_l2_reverse
hid2out.weight
hid2out.bias


In [0]:
import torch.optim as optim

In [0]:
eclassifier = eclassifier.to(device)

In [0]:
opt = optim.Adam(eclassifier.parameters())
crit = nn.BCELoss().to(device)

In [0]:
def accuracy(out, trg):
  ind = out.argmax(dim = 1)
  correct = (ind == trg).float()
  return correct.sum() / len(correct)

In [0]:
def lbl2trg(lbl):
  trg = torch.zeros((len(lbl), len(LABEL.vocab))).to(device)
  for i in range(len(lbl)):
    trg[i][int(lbl[i])] = 1
  return trg

In [0]:
def train(eclassifier, it, opt, crit):
  epoch_loss = 0
  epoch_acc = 0

  eclassifier.train()

  for batch in it:
    opt.zero_grad()
    out = eclassifier(batch.premise, batch.hypothesis).squeeze(1)
    
    loss = crit(out, lbl2trg(batch.label))
    acc = accuracy(out, batch.label)
    
    loss.backward()
    opt.step()
    
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  
  return epoch_loss / len(it), epoch_acc / len(it)

In [0]:
def eval(eclassifier, it, crit):
  epoch_loss = 0
  epoch_acc = 0

  eclassifier.eval()

  with torch.no_grad():
    for batch in it:
      out = eclassifier(batch.premise, batch.hypothesis).squeeze(1)
      
      loss = crit(out, lbl2trg(batch.label))
      acc = accuracy(out, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()
  
  return epoch_loss / len(it), epoch_acc / len(it)

In [0]:
import time

In [0]:
def epoch_time(start, end):
  delta = end - start
  delta_min = int(delta / 60)
  delta_sec = int(delta - delta_min * 60)
  return delta_min, delta_sec

In [0]:
best = float('inf')

In [0]:
EPOCH_NUM = 5

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(eclassifier, train_it, opt, crit)
  val_loss, val_acc = eval(eclassifier, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(eclassifier.state_dict(), '/content/gdrive/My Drive/Models/eclassifier_save.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Val (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 51 m 25 s
Train (loss | acc): 0.31657142626106016 | 0.7944796139518457
Val (loss | acc): 0.31260934375323257 | 0.7959155703519846
Epoch: 1
Time: 51 m 31 s
Train (loss | acc): 0.29643368118990276 | 0.8091571274919146
Val (loss | acc): 0.35581663347683945 | 0.766657410503982
Epoch: 2
Time: 51 m 24 s
Train (loss | acc): 0.28413346101497783 | 0.8185562196171183
Val (loss | acc): 0.3167535966479933 | 0.7897264469753612
Epoch: 3
Time: 51 m 24 s
Train (loss | acc): 0.27430675441932145 | 0.8257537414454347
Val (loss | acc): 0.29538559449183477 | 0.8080143541484684
Epoch: 4
Time: 51 m 32 s
Train (loss | acc): 0.2676892052339485 | 0.8307199057639478
Val (loss | acc): 0.298282051241243 | 0.8066829001748717


In [0]:
print('Epoch: ' + str(i))
print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
print('Val (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 51 m 31 s
Train (loss | acc): 0.3764739676629508 | 0.7402369174463995
Val (loss | acc): 0.34423330464920443 | 0.7764866715901858


In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(eclassifier, train_it, opt, crit)
  val_loss, val_acc = eval(eclassifier, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(eclassifier.state_dict(), '/content/gdrive/My Drive/Models/eclassifier.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Val (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 22 m 39 s
Train (loss | acc): 0.2673197421284335 | 0.8312675347977004
Val (loss | acc): 0.298156807755495 | 0.8063660573649716
Epoch: 1
Time: 22 m 40 s
Train (loss | acc): 0.2616935626769444 | 0.8354392755787059
Val (loss | acc): 0.3029975707267786 | 0.8027775403741119
Epoch: 2
Time: 22 m 39 s
Train (loss | acc): 0.2566583293306972 | 0.8388725929691319
Val (loss | acc): 0.2938473594266099 | 0.8195559924299066
Epoch: 3
Time: 22 m 41 s
Train (loss | acc): 0.253077296632145 | 0.8408710251927709
Val (loss | acc): 0.2879910020085124 | 0.8208874456294171
Epoch: 4
Time: 22 m 43 s
Train (loss | acc): 0.24963635965862346 | 0.8437747833668797
Val (loss | acc): 0.28269058059562335 | 0.8215852132091275


In [0]:
best, _ = eval(eclassifier, val_it, crit)
print(best, _)

0.28269058059562335 0.8215852132091275


In [0]:
eclassifier.load_state_dict(torch.load('/content/gdrive/My Drive/Models/eclassifier.pt'))

<All keys matched successfully>

In [0]:
test_loss, test_acc = eval(eclassifier, test_it, crit)

In [0]:
print(test_loss, test_acc)

0.28997930923065585 0.8158482142857143


In [0]:
def eclassify(eclassifier, tokenizer, pre, hyp):
  eclassifier.eval()
  pre_toks = tokenizer.tokenize(pre)[:tokenizer.max_model_input_sizes['bert-base-uncased'] - 2]
  hyp_toks = tokenizer.tokenize(hyp)[:tokenizer.max_model_input_sizes['bert-base-uncased'] - 2]
  
  pre_inds = [CLS] + tokenizer.convert_tokens_to_ids(pre_toks) + [SEP]
  hyp_inds = tokenizer.convert_tokens_to_ids(pre_toks)
  
  pre_inp = torch.LongTensor(pre_inds).unsqueeze(0).to(device)
  hyp_inp = torch.LongTensor(hyp_inds).unsqueeze(0).to(device)
  
  return LABEL.vocab.itos[eclassifier(pre_inp, hyp_inp).argmax(dim = 1).item()]

In [0]:
def eclassify_list(eclassifier, tokenizer, prehyps):
  outs = []
  for (pre, hyp) in prehyps:
    outs.append(eclassify(eclassifier, tokenizer, pre, hyp))
  return outs

In [0]:
eclassify_list(eclassifier, tokenizer, [("Someone is playing piano", "The music sonds nice"), ("Public transport is overcrowded", "Weather is pretty good"), ("It's boring", "I'm absolutely delighted")])

['entailment', 'neutral', 'contradiction']

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(eclassifier, train_it, opt, crit)
  val_loss, val_acc = eval(eclassifier, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(eclassifier.state_dict(), '/content/gdrive/My Drive/Models/eclassifier.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Val (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 22 m 39 s
Train (loss | acc): 0.24712767098567692 | 0.8459280692075639
Val (loss | acc): 0.29497533649593205 | 0.815193167754582
Epoch: 1
Time: 22 m 41 s
Train (loss | acc): 0.24457184616944996 | 0.8475317067891109
Val (loss | acc): 0.2922282685320099 | 0.8161953176770892
Epoch: 2
Time: 22 m 36 s
Train (loss | acc): 0.24293427207513268 | 0.8487007696275276
Val (loss | acc): 0.2930526646313729 | 0.8167791637507352
Epoch: 3
Time: 22 m 41 s
Train (loss | acc): 0.2419142763079157 | 0.8496030151899574
Val (loss | acc): 0.2893963125231978 | 0.820785984590456
Epoch: 4
Time: 22 m 37 s
Train (loss | acc): 0.24036355849914426 | 0.8505910113662176
Val (loss | acc): 0.29008466624594353 | 0.8200508371576086


5 layers

In [0]:
HID_DIM = 256
OUT_DIM = len(LABEL.vocab)
NUM_LAY = 5
BI = True
DROP = 0.25

eclassifier_5 = EClassifier(bert, HID_DIM, OUT_DIM, NUM_LAY, BI, DROP)

In [0]:
eclassifier_save = eclassifier_5.to(device)

In [0]:
opt = optim.Adam(eclassifier_5.parameters())
crit = nn.BCELoss().to(device)

In [0]:
best = float('inf')

In [0]:
EPOCH_NUM = 5

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(eclassifier_5, train_it, opt, crit)
  val_loss, val_acc = eval(eclassifier_5, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(eclassifier_5.state_dict(), '/content/gdrive/My Drive/Models/eclassifier_5.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Val (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 25 m 39 s
Train (loss | acc): 0.3798838868973306 | 0.7371019879069599
Val (loss | acc): 0.39786947721784766 | 0.7138478299240013
Epoch: 1
Time: 25 m 39 s
Train (loss | acc): 0.3189881232264213 | 0.7931386888693785
Val (loss | acc): 0.3253915145412668 | 0.7901839118499261
Epoch: 2
Time: 25 m 38 s
Train (loss | acc): 0.2976155682065058 | 0.8095778035206541
Val (loss | acc): 0.3199304704155241 | 0.7869869989234132
Epoch: 3
Time: 25 m 39 s
Train (loss | acc): 0.2844476982715079 | 0.8192044928433844
Val (loss | acc): 0.3161048233122021 | 0.7893579834467405
Epoch: 4
Time: 25 m 41 s
Train (loss | acc): 0.27538838870094295 | 0.8254757742401827
Val (loss | acc): 0.29896490198451203 | 0.8118698736289879


In [0]:
eclassifier_5.load_state_dict(torch.load('/content/gdrive/My Drive/Models/eclassifier_5.pt'))

<All keys matched successfully>

In [0]:
best, _ = eval(eclassifier_5, val_it, crit)
print(best, _)

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(eclassifier_5, train_it, opt, crit)
  val_loss, val_acc = eval(eclassifier_5, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(eclassifier_5.state_dict(), '/content/gdrive/My Drive/Models/eclassifier_5.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Val (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

KeyboardInterrupt: ignored